- Title: Hands on the Python module subprocess
- Slug: hands-on-the-python-model-subprocess
- Date: 2019-10-19
- Category: Programming
- Tags: programming, Python, subprocess
- Author: Ben Du

## General Tips

1. The method `subprocess.run` is preferred over the older high-level APIs 
    (`subprocess.call`, `subprocess.check_call` and `subprocess.check_output`).
    The method `subprocess.Popen` (which powers the high-level APIs) can be used if you need advanced control.
 
2. Avoid using system shell (`shell=True`) for security reasons.

3. There are at least 2 advantages of passing a shell command as a list. 
    First, avoid shell injection attack.
    Second, there is no need to you to manually escape special characters in the command.
    
4. It is suggested that you keep the option `check=False` 
    as error code is more flexible than throwing exception in subprocess
    (even though throwing exception is preferred to error code generally speaking).

## Capture the Standard Ouput and Error

In Python 3.7+, 
the output (stdout and stderr) of commands can be captured 
by specifying the option `capture_output=True`.
This option is equivalent to the options `stdout=PIPE, stderr=PIPE` in older versions of Python.

Capture stdout by specifying `stdout=sp.PIPE`.

In [23]:
process = sp.run(['pwd'], stdout=sp.PIPE, stderr=sp.PIPE)
print(process.stdout)

b'/app/archives/blog/misc/content\n'


Capture both the standard ouput and error (separately).

In [26]:
process = sp.run(['pwd', '-l'], stdout=sp.PIPE, stderr=sp.PIPE)
print(process.stdout)
print(process.stderr)

b''
b"pwd: invalid option -- 'l'\nTry 'pwd --help' for more information.\n"


Capture both the standard output and error in one place (`process.stdout`).

In [27]:
process = sp.run(['pwd', '-l'], stdout=sp.PIPE, stderr=sp.STDOUT)
print(process.stdout)

b"pwd: invalid option -- 'l'\nTry 'pwd --help' for more information.\n"


## Supress the Output of `subprocess.run`

To suppress the output of `subprocess.run`,
    you can redirect the output to `/dev/null`.

In [13]:
import os
import subprocess as sp
import sys

Without redicting the standard output to `/dev/null` 
(i.e., supressing the standard output), 
the command outputs results. 
(Note that there is bug in ipykernel which supress the output. 
This comamnd outputs results in a regular Python shell.)

In [8]:
sp.run(['ls', '-l'])

CompletedProcess(args=['ls', '-l'], returncode=0)

After redirecting the standard output to `/dev/null` 
(i.e., supressing the standard output), 
the command does not output any result.

In [1]:
with open(os.devnull, 'w') as devnull:
    sp.run(['ls', '-l'], stdout=devnull)

The code below supress both the stdout and stderr 
by redirecting both of them to `/dev/null`.

In [9]:
with open(os.devnull, 'w') as devnull:
    sp.run(['ls', '-l'], stdout=devnull, stderr=devnull)

Below is an equivalent approach,
which merges stderr to stdout first 
and then redirect stdout to `/dev/null`.

In [11]:
with open(os.devnull, 'w') as devnull:
    sp.run(['ls', '-l'], stdout=devnull, stderr=sp.STDOUT)

## Comparison of Differenct Devices

1. `sys.stdout` is the standard output stream.
  `subprocess.STDOUT` refers to the standard out stream of subprocess.
  It is either `subprocess.PIPE` or `None`.
  
    :::python
    os.devnull
    subprocess.DEVNULL
    with open(os.devnull, 'w') as devnull:
        pass

## Possible Exceptions

### FileNotFoundError

If the command is not found, 
`subprocess.run` throws the exception `FileNotFoundError` (even `check=False`).

### subprocess.CalledProcessError

If the command fails to run
and `check=True`,
`subprocess.run` throws the exception `subprocess.CalledProcessError`.

## Issues in JupyterLab Notebooks

Running `sp.run('ls -a')` in a JupyterLab notebook prints nothing 
while running it in a regular Python shell prints results.
This is likely a bug in ipykernel.